In [4]:
import os
import sys
import warnings
import pandas as pd
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
from tradingviewstrategy import TradingViewStrategy
from data_manipulation import DataManipulation

In [6]:
screener="turkey"
exchange="BIST"
path_symbol = './symbol_data/ana_market.csv'
index = 'Order'

st_main_market = TradingViewStrategy(exchange, screener, path_symbol)
filter_list = st_main_market.get_st_result_list(interval='1d')

In [7]:
len(filter_list)

76

In [8]:
SYMBOL = ''
scale = 1
range = [5, 8, 10, 15, 20, 25]
range = [i*scale for i in range]
period = '1y'
interval = '1d'
start_date = ''
end_data = ''
source = 'yahoo'

In [ ]:
df_filter_list = []
for i in filter_list:
    SYMBOL = f'{i}.IS'
    data = DataManipulation(SYMBOL, source, range, period=period, interval=interval, scale=scale, prefix_path='../ana_market')
    if data.df is not None:
        df_filter_list.append(i)

In [22]:
len(df_filter_list)

66